<a href="https://colab.research.google.com/github/SpellOnYou/100_Days_of_ML_Code/blob/master/code/Neural_Network_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [0]:
sents_kor = ["나는 강아지를 좋아한다", "나는 커피를 사랑한다", "나는 우유를 싫어한다"]
sents_eng = ["i like dog", "i love coffee", "i hate milk", ]

In [0]:
words_kor = list(set(" ".join(sents_kor).split()))
words_eng = list(set(" ".join(sents_eng).split()))

In [0]:
def word_dict(word_list, idx = False):
  if not idx:
    return{word:idx for idx, word in enumerate(word_list)}
  return {idx:word for idx, word in enumerate(word_list)}  

In [0]:
bag_kor = word_dict(words_kor)
bag_eng = word_dict(words_eng)
n_class_eng = len(bag_eng)
n_class_kor = len(bag_kor)

In [56]:
len(bag_kor)

7

In [0]:
# NNLM Parametber
n_step = 2 # number of steps ['i like', 'like dog']
n_hidden = 1 # number of hidden units

In [0]:
def make_batch(sentences):
  input_batch, target_batch = [], []
  for sent in sentences:
    words = sent.split()
    input = [bag_kor[w] for w in words[:-1]] #except the last word, list of idx
#     print(input)
    target = bag_kor[words[-1]] #last word, idx
#     print(target)
    
    # one-hot encoding
    input_batch.append(np.eye(n_class_kor)[input])
    target_batch.append(np.eye(n_class_kor)[target])
  return input_batch, target_batch

In [61]:
make_batch(sents_kor)

([array([[0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1.]]), array([[0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.]]), array([[0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.]])],
 [array([0., 0., 0., 0., 1., 0., 0.]),
  array([0., 0., 1., 0., 0., 0., 0.]),
  array([1., 0., 0., 0., 0., 0., 0.])])

In [0]:
X = tf.placeholder(tf.float32, [None, n_step, n_class_kor]) # [batch_size, number of steps, number of Vocabulary]
Y = tf.placeholder(tf.float32, [None, n_class_kor])

[batch_size, number of steps, number of Vocabulary]

In [66]:
input = tf.reshape(X, shape=[-1, n_step * n_class_kor])

Instructions for updating:
Colocations handled automatically by placer.


#### H, hidden layer, matrix size:  (n_step * n_class_kor, n_hidden)

In [75]:
H = tf.Variable(tf.random_normal([n_step * n_class_kor, n_hidden])); H

<tf.Variable 'Variable_6:0' shape=(14, 1) dtype=float32_ref>

#### b, Bias of hidden layer

In [70]:
d = tf.Variable(tf.random_normal([n_hidden])); d

<tf.Variable 'Variable_3:0' shape=(1,) dtype=float32_ref>

In [72]:
U = tf.Variable(tf.random_normal([n_hidden, n_class_kor])); U

<tf.Variable 'Variable_4:0' shape=(1, 7) dtype=float32_ref>

In [74]:
b = tf.Variable(tf.random_normal([n_class_kor])); b

<tf.Variable 'Variable_5:0' shape=(7,) dtype=float32_ref>

In [0]:
tanh = tf.nn.tanh(d + tf.matmul(input, H)) # [batch_size, n_hidden]
model = tf.matmul(tanh, U) + b # [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
prediction =tf.argmax(model, 1)

### Training

In [77]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(sents_kor)

for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

Epoch: 1000 cost = 1.554112
Epoch: 2000 cost = 1.189193
Epoch: 3000 cost = 1.021053
Epoch: 4000 cost = 0.558948
Epoch: 5000 cost = 0.340857


### Predict

In [0]:
predict =  sess.run([prediction], feed_dict={X: input_batch})

In [79]:
predict

[array([4, 2, 0])]

### Test

In [0]:
input = [sen.split()[:2] for sen in sents_kor]

In [82]:
input

[['나는', '강아지를'], ['나는', '커피를'], ['나는', '우유를']]

In [0]:
number_dict_kor = word_dict(words_kor, idx=True)

In [86]:
print([sen.split()[:2] for sen in sents_kor], '->', [number_dict_kor[n] for n in predict[0]])

[['나는', '강아지를'], ['나는', '커피를'], ['나는', '우유를']] -> ['좋아한다', '사랑한다', '싫어한다']
